## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-11-20-50-22 +0000,nypost,"Texas Target shooting leaves 2 dead, several o...",https://nypost.com/2025/08/11/us-news/texas-ta...
1,2025-08-11-20-50-03 +0000,nyt,"Who is Lip-Bu Tan, the Intel CEO in Trump’s Cr...",https://www.nytimes.com/2025/08/11/technology/...
2,2025-08-11-20-49-43 +0000,nyt,Israel’s Plan for Gaza City Is Still Unclear,https://www.nytimes.com/2025/08/11/world/middl...
3,2025-08-11-20-49-43 +0000,nyt,Harvard Nears a Deal With the Trump Administra...,https://www.nytimes.com/2025/08/11/us/trump-ha...
4,2025-08-11-20-45-45 +0000,nypost,Mamdani launches Trump-hating tour across 5 bo...,https://nypost.com/2025/08/11/us-news/socialis...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2311/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
15,trump,52
70,china,16
59,he,15
217,new,11
75,al,10


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
29,2025-08-11-19-43-00 +0000,wsj,The Trump administration’s decision to charge ...,https://www.wsj.com/politics/national-security...,128
17,2025-08-11-20-00-00 +0000,wsj,President Trump said he would deploy National ...,https://www.wsj.com/politics/policy/trump-dc-n...,121
9,2025-08-11-20-35-03 +0000,startribune,Trump says he's placing Washington police unde...,https://www.startribune.com/trump-is-promising...,107
105,2025-08-11-16-09-47 +0000,nypost,Trump says he’ll tell Putin to end Ukraine inv...,https://nypost.com/2025/08/11/world-news/trump...,106
224,2025-08-11-01-00-00 +0000,wsj,News the jobs market was much weaker than prev...,https://www.wsj.com/economy/jobs-inflation-tru...,106


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
29,128,2025-08-11-19-43-00 +0000,wsj,The Trump administration’s decision to charge ...,https://www.wsj.com/politics/national-security...
17,61,2025-08-11-20-00-00 +0000,wsj,President Trump said he would deploy National ...,https://www.wsj.com/politics/policy/trump-dc-n...
19,49,2025-08-11-19-58-17 +0000,nyt,Who Were the Al Jazeera Journalists Killed in ...,https://www.nytimes.com/2025/08/11/world/middl...
101,49,2025-08-11-16-34-28 +0000,nypost,"At least one dead, dozens trapped or injured a...",https://nypost.com/2025/08/11/us-news/people-t...
52,39,2025-08-11-18-53-01 +0000,startribune,Byron Buxton will lead off Twins game tonight ...,https://www.startribune.com/twins-yankees-seri...
143,38,2025-08-11-13-46-00 +0000,wsj,Colombian Presidential Hopeful Dies Two Months...,https://www.wsj.com/world/americas/colombian-p...
224,35,2025-08-11-01-00-00 +0000,wsj,News the jobs market was much weaker than prev...,https://www.wsj.com/economy/jobs-inflation-tru...
229,32,2025-08-10-23-18-32 +0000,nypost,Intel CEO Lip-Bu Tan to visit White House on M...,https://nypost.com/2025/08/10/business/intel-c...
144,32,2025-08-11-13-34-29 +0000,nypost,Nuclear winter would bring mass starvation and...,https://nypost.com/2025/08/11/science/nuclear-...
121,30,2025-08-11-15-02-46 +0000,nypost,Wild video shows Chinese ships crashing while ...,https://nypost.com/2025/08/11/world-news/chine...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
